### Task evaluation

#### Loading Models
Please note the models output `task,domain,feature`

In [ ]:
! pip install pandas

In [16]:
import torch
from pathlib import Path
import pandas as pd

import torch.nn as nn

from data_config import DataConfig
from model_DANN import DANN
from ds_sfew import DatasetSFEW
from ds_expw import EXPW
from run_config import RunConfig


In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


#### Loading Models to evaluate

In [5]:
dataconfig_inference = DataConfig() 

# NON DANN
non_dann_model_sfew_name = 'non_dann_sfew_12_12.pt'
non_dann_model_inference = DANN(num_classes=7) # skeleton copy
NON_DANN_SFEW_DIR = dataconfig_inference.NON_DANN_SFEW_DIR # Path(self.MODEL_DIR,"non_dann_sfew")

try:
    non_dann_model_inference.load_state_dict(torch.load(Path(NON_DANN_SFEW_DIR, non_dann_model_sfew_name),
                                                        map_location=torch.device(device)))
    print(f'{non_dann_model_sfew_name} locked and loaded')
except:
    print(f' problem in loading {non_dann_model_sfew_name}')

# DANN
dann_model_sfew_expw_name = 'dann_sfew_expw_20_12.pt'
dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
DANN_SFEW_EXPW_DIR = dataconfig_inference.DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"dann_sfew_expw")
try:
    dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(DANN_SFEW_EXPW_DIR, dann_model_sfew_expw_name),
                                                              map_location=torch.device(device)))
    print(f'{dann_model_sfew_expw_name} locked and loaded')
except:
    print(f' problem in loading {dann_model_sfew_expw_name}')

# EWC DANN
ewc_dann_model_sfew_expw_name = 'ewc_dann_sfew_expw_20_12.pt'
ewc_dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
EWC_DANN_SFEW_EXPW_DIR = dataconfig_inference.EWC_DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"ewc_dann_sfew_expw")
try:
    ewc_dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(EWC_DANN_SFEW_EXPW_DIR, ewc_dann_model_sfew_expw_name),
                                                              map_location=torch.device(device)))
    print(f'{ewc_dann_model_sfew_expw_name} locked and loaded')
except:
    print(f' problem in loading {ewc_dann_model_sfew_expw_name}')   


non_dann_sfew_12_12.pt locked and loaded
dann_sfew_expw_20_12.pt locked and loaded


### Dataloaders for source and target

In [9]:
sfew = DatasetSFEW()
sfew_train_loader, sfew_val_loader = sfew.get_dataloader()

self.BASE_PATH -dataset,self.EXTRACT_DIR-dataset\sfew,self.ZIP_FILE_PATH - dataset\SFEW_2.zip 
Directory dataset\sfew already exists.
Files exist in dataset\sfew, extraction not done
Unzipped Files already exist in dataset\sfew\Train, not extracted
Unzipped Files already exist in dataset\sfew\Val, not extracted


In [10]:
expw_obj = EXPW()
expw_train_ds, expw_val_ds = expw_obj.get_dataset()
expw_train_loader, expw_val_loader = expw_obj.get_dataloader()

Directory dataset\expwds already exists.
Directory dataset\expwds already exists.


#### Printing comparison

In [15]:
#@title Typical Test Function
#TODO : move this to common file

def get_correct_predictions(prediction, labels):
    """
    Function to return total number of correct predictions
    :param prediction: Model predictions on a given sample of data
    :param labels: Correct labels of a given sample of data
    :return: Number of correct predictions
    """
    return prediction.argmax(dim=1).eq(labels.argmax(dim=1)).sum().item()

def test_model(model, device, data_loader, mode= 'test',
               class_loss_function = nn.CrossEntropyLoss() ):
    model.eval() # evaluation mode

    # variables to track loss and accuracy
    test_loss = 0.0
    correct = 0

    # Disable gradient updation
    with torch.no_grad():
        # Iterate over each batch and fetch images and labels from the batch
        for batch_idx, batch in enumerate(data_loader):
            images, labels = batch[0].to(device), batch[1].to(device)
            
            class_output, _ , _ = model(images) # forward pass, output = {class, domain, feature}
            class_loss = class_loss_function(class_output,labels) # Calculate Loss

            test_loss += class_loss.item() # sum up batch loss

            # Sum up batch correct predictions
            correct += get_correct_predictions(class_output, labels)

    # Calculate test loss for a epoch
    test_loss /= len(data_loader.dataset)
    accuracy = 100. * correct / len(data_loader.dataset)

    print('Test / Val: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(data_loader.dataset),
        accuracy))
    

    return correct, test_loss, accuracy



In [18]:
runconfig = RunConfig()
_,_,non_dann_accuracy_source = test_model(model =non_dann_model_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,non_dann_accuracy_target = test_model(model =non_dann_model_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,dann_accuracy_source = test_model(model =dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,dann_accuracy_target = test_model(model =dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,ewc_dann_accuracy_source = test_model(model =ewc_dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,ewc_dann_accuracy_target = test_model(model =ewc_dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )


Test / Val: Average loss: 0.4706, Accuracy: 96/436 (22.02%)

Test / Val: Average loss: 0.4963, Accuracy: 2071/13620 (15.21%)

Test / Val: Average loss: 0.4934, Accuracy: 104/436 (23.85%)

Test / Val: Average loss: 0.4948, Accuracy: 2138/13620 (15.70%)



In [29]:
# ewc_dann_model_sfew_expw_name = 'ewc_dann_sfew_expw_20_12.pt'
# ewc_dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
# EWC_DANN_SFEW_EXPW_DIR = dataconfig_inference.EWC_DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"ewc_dann_sfew_expw")
# try:
#     ewc_dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(EWC_DANN_SFEW_EXPW_DIR, ewc_dann_model_sfew_expw_name),
#                                                               map_location=torch.device(device)))
#     print(f'{ewc_dann_model_sfew_expw_name} locked and loaded')
# except:
#     print(f' problem in loading {ewc_dann_model_sfew_expw_name}')   

# _,_,ewc_dann_accuracy_source = test_model(model =ewc_dann_model_sfew_expw_inference,
#                                  device=device,
#                                  data_loader=sfew_val_loader,
#                                  class_loss_function=runconfig.criterion_class )

# _,_,ewc_dann_accuracy_target = test_model(model =ewc_dann_model_sfew_expw_inference,
#                                  device=device,
#                                  data_loader=expw_val_loader,
#                                  class_loss_function=runconfig.criterion_class )

ewc_dann_sfew_expw_20_12.pt locked and loaded
Test / Val: Average loss: 0.4963, Accuracy: 102/436 (23.39%)

Test / Val: Average loss: 0.4866, Accuracy: 2256/13620 (16.56%)



In [31]:
source= 'sfew'
target = 'expw'

col_name = ['model_class','trained_on','accuracy_source','accuracy_target']
df_nice_results = pd.DataFrame(columns = col_name)
print(df_nice_results)
dict_input = {}
dict_input = {'model_class': 'Non_DANN', 
                   'trained_on': str(source)+ " source",
                #    'accuracy_source':dict_non_dann_losses_list_sfew_expw['test_accuracy_source'],
                #    'accuracy_target':dict_non_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':non_dann_accuracy_source,
                   'accuracy_target':non_dann_accuracy_target
                   }
print(dict_input)
df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


dict_input = {'model_class': 'DANN', 
                   'trained_on': str(source)+" source + "+str(target) + " target",
                  #  'accuracy_source':dict_dann_losses_list_sfew_expw['test_accuracy_source'],
                  #  'accuracy_target':dict_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':dann_accuracy_source,
                   'accuracy_target':dann_accuracy_target
                   }
print(dict_input)

df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


dict_input = {'model_class': 'EWC_DANN', 
                   'trained_on': str(source)+" source + "+str(target) + " target",
                  #  'accuracy_source':dict_ewc_dann_losses_list_sfew_expw['test_accuracy_source'],
                  #  'accuracy_target':dict_ewc_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':ewc_dann_accuracy_source,
                   'accuracy_target':ewc_dann_accuracy_target
                   }

df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


df_nice_results

Empty DataFrame
Columns: [model_class, trained_on, accuracy_source, accuracy_target]
Index: []
{'model_class': 'Non_DANN', 'trained_on': 'sfew source', 'accuracy_source': 22.01834862385321, 'accuracy_target': 15.205580029368576}
{'model_class': 'DANN', 'trained_on': 'sfew source + expw target', 'accuracy_source': 23.853211009174313, 'accuracy_target': 15.697503671071953}


C:\Users\Admin\AppData\Local\Temp\ipykernel_14332\4160264040.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


,model_class,trained_on,accuracy_source,accuracy_target
0,Non_DANN,sfew source,22.018349,15.205580
1,DANN,sfew source + expw target,23.853211,15.697504
2,EWC_DANN,sfew source + expw target,23.394495,16.563877


### Bias Evaluation